In [1]:
import numpy as np
from scipy import linalg
from scipy.spatial import distance

import copy
import os 
import gc 
import pickle

import torch
from torch import nn, optim, autograd
import torch.nn.functional as F
import torch.nn.init as init
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

import torch.utils.data as data

from src.utils import *
from src.client import *
from src.clustering import *
from src.data import *
from src.models import *

import matplotlib.pyplot as plt
import numpy as np
import matplotlib 
matplotlib.use('nbagg')
import pylab
from matplotlib.pyplot import subplots
import pickle 
import pandas as pd

import seaborn as sns

In [3]:
class Args: 
    num_users = 100
    seed = 1
    gpu = 1
    
    ## CIFAR-10 has 50000 training images (5000 per class), 10 classes, 10000 test images (1000 per class)
    ## CIFAR-100 has 50000 training images (500 per class), 100 classes, 10000 test images (100 per class)
    ## MNIST has 60000 training images (min: 5421, max: 6742 per class), 10000 test images (min: 892, max: 1135
    ## per class) --> in the code we fixed 5000 training image per class, and 900 test image per class to be 
    ## consistent with CIFAR-10 
    
    ## CIFAR-10 Non-IID 250 samples per label for 2 class non-iid is the benchmark (500 samples for each client)
    
    nsample_pc = 250  ## number of samples per class for each client 
    nclass = 2        ## number of classes or shards for each client
    model = 'simple-cnn' ## options: lenet5
    dataset = 'fmnist'  ## options: mnist, cifar10, cifar100
    datadir = '../data/'
    logdir = '../logs/'
    partition = 'noniid-#label2'
    alg = 'pacfl'
    savedir = '../save/'
    beta = 0.1
    local_view = True
    batch_size= 10
    noise = 0
    noise_type = 'level'
    
    rounds = 200
    frac = 0.1
    local_bs = 10
    local_ep = 10
    lr = 0.01
    momentum = 0.5
    
    cluster_alpha = 3.5
    nclasses = 10 
    nsamples_shared = 2500
    n_basis = 3
    linkage = 'average'
   
    print_freq = 50
    
    load_initial = ''
    
args = Args()

torch.cuda.set_device(args.gpu) ## Setting cuda on GPU 
#torch.manual_seed(args.seed)
#np.random.seed(args.seed)

args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() else 'cpu')

In [4]:
args.dataset = 'cifar10'

In [5]:

args.local_view = True
X_train, y_train, X_test, y_test, net_dataidx_map, net_dataidx_map_test, \
traindata_cls_counts, testdata_cls_counts = partition_data(args.dataset, 
args.datadir, args.logdir, args.partition, args.num_users, beta=args.beta, local_view=args.local_view)

train_dl_global, test_dl_global, train_ds_global, test_ds_global = get_dataloader(args.dataset,
                                                                                   args.datadir,
                                                                                   args.batch_size,
                                                                                   32)

print("len train_ds_global:", len(train_ds_global))
print("len test_ds_global:", len(test_ds_global))
print(args.dataset)

Files already downloaded and verified
Files already downloaded and verified
K: 10
partition: noniid-#label2
Data statistics Train:
 {0: {0: 228, 6: 239}, 1: {1: 278, 7: 295}, 2: {2: 278, 7: 295}, 3: {3: 239, 4: 239}, 4: {4: 239, 7: 294}, 5: {5: 239, 9: 264}, 6: {6: 239, 8: 228}, 7: {2: 278, 7: 294}, 8: {3: 239, 8: 228}, 9: {5: 239, 9: 264}, 10: {0: 228, 5: 238}, 11: {1: 278, 5: 238}, 12: {2: 278, 8: 228}, 13: {3: 238, 4: 238}, 14: {0: 228, 4: 238}, 15: {0: 228, 5: 238}, 16: {0: 228, 6: 238}, 17: {7: 294, 8: 228}, 18: {7: 294, 8: 228}, 19: {0: 228, 9: 264}, 20: {0: 227, 6: 238}, 21: {1: 278, 4: 238}, 22: {2: 278, 3: 238}, 23: {0: 227, 3: 238}, 24: {3: 238, 4: 238}, 25: {0: 227, 5: 238}, 26: {6: 238, 7: 294}, 27: {7: 294, 8: 228}, 28: {1: 278, 8: 227}, 29: {6: 238, 9: 263}, 30: {0: 227, 8: 227}, 31: {1: 278, 7: 294}, 32: {2: 278, 5: 238}, 33: {3: 238, 8: 227}, 34: {0: 227, 4: 238}, 35: {2: 278, 5: 238}, 36: {6: 238, 9: 263}, 37: {7: 294, 8: 227}, 38: {8: 227, 9: 263}, 39: {2: 278, 9: 263

In [6]:
idxs_train = np.arange(len(train_ds_global))
labels_train = np.array(train_ds_global.target)
# Sort Labels Train 
idxs_labels_train = np.vstack((idxs_train, labels_train))
idxs_labels_train = idxs_labels_train[:, idxs_labels_train[1, :].argsort()]
idxs_train = idxs_labels_train[0, :]
labels_train = idxs_labels_train[1, :]

In [7]:
# tinydata = [np.asarray(img).astype('float32') for img in train_ds_global.data]

In [8]:
def hierarchical_clustering(A, thresh=1.5, linkage='maximum'):
    '''
    Hierarchical Clustering Algorithm. It is based on single linkage, finds the minimum element and merges
    rows and columns replacing the minimum elements. It is working on adjacency matrix. 
    
    :param: A (adjacency matrix), thresh (stopping threshold)
    :type: A (np.array), thresh (int)
    
    :return: clusters
    '''
    label_assg = {i: i for i in range(A.shape[0])}
    
    B = copy.deepcopy(A)
    step = 0
    while A.shape[0] > 1:
        np.fill_diagonal(A,-np.NINF)
        #print(f'step {step} \n {A}')
        step+=1
        ind=np.unravel_index(np.argmin(A, axis=None), A.shape)
        
        if A[ind[0],ind[1]]>thresh:
            print('Breaking HC')
            #print(f'A {B}')
            break
        else:
            np.fill_diagonal(A,0)
            if linkage == 'maximum':
                Z=np.maximum(A[:,ind[0]], A[:,ind[1]])
            elif linkage == 'minimum':
                Z=np.minimum(A[:,ind[0]], A[:,ind[1]])
            elif linkage == 'average':
                Z= (A[:,ind[0]] + A[:,ind[1]])/2
            
            A[:,ind[0]]=Z
            A[:,ind[1]]=Z
            A[ind[0],:]=Z
            A[ind[1],:]=Z
            A = np.delete(A, (ind[1]), axis=0)
            A = np.delete(A, (ind[1]), axis=1)
            
            B = copy.deepcopy(A)
            if type(label_assg[ind[0]]) == list: 
                label_assg[ind[0]].append(label_assg[ind[1]])
            else: 
                label_assg[ind[0]] = [label_assg[ind[0]], label_assg[ind[1]]]

            label_assg.pop(ind[1], None)

            temp = []
            for k,v in label_assg.items():
                if k > ind[1]: 
                    kk = k-1
                    vv = v
                else: 
                    kk = k 
                    vv = v
                temp.append((kk,vv))

            label_assg = dict(temp)

    clusters = []
    for k in label_assg.keys():
        if type(label_assg[k]) == list:
            clusters.append(list(flatten(label_assg[k])))
        elif type(label_assg[k]) == int: 
            clusters.append([label_assg[k]])
            
    #print(label_assg)
            
    return clusters

In [9]:
def Eq_Basis(A,B):
    AB=np.arccos(A.T@B)
    A_E=np.zeros((A.shape[0],A.shape[1]))
    B_E=np.zeros((B.shape[0],B.shape[1]))
    for i in range(AB.shape[0]):
        ind = np.unravel_index(np.argmin(AB, axis=None), AB.shape)
        AB[ind[0],:]=AB[:,ind[1]]=0
        A_E[:,i]=A[:,ind[0]]
        B_E[:,i]=B[:,ind[1]]
    return  A_E,B_E

In [10]:
from numpy import linalg as LA

def Gau_Lin_ker(A,B,kernel=None):

    Sig=np.zeros((A.shape[1],B.shape[1]))
    Ker_G=np.zeros((A.shape[1],B.shape[1]))
    Ker_L=np.zeros((A.shape[1],B.shape[1]))

    for i in range(A.shape[1]):
        for j in range(B.shape[1]):
            Sig [i,j]= LA.norm(A[:,i]-B[:,j], 2)**2
    sigma=Sig.mean()

    for i in range(A.shape[1]):
        for j in range(B.shape[1]):
            Ker_G[i,j]=np.exp(-LA.norm(A[:,i]-B[:,j], 1)**0.9/(2*sigma))
            Ker_L[i,j]=np.dot(A[:,i],B[:,j])
   
    if kernel == "Gaussian":
        return Ker_G
    elif kernel == "Linear":
        return Ker_L

## SVD Per Class 

In [11]:
data_per_class = {}
U_per_class = {}
K = 3
nclass=10
for i in range(nclass):
    print(f'Class {i}')
    inds = np.where(labels_train==i)[0]
    idx = idxs_train[inds].astype(int)
    length = len(idx)

    #length = 500
    data_per_class[i] = np.array(train_ds_global.data[idx]).reshape(length, -1).T.astype(float)
    
    u1_temp, sh1_temp, vh1_temp = np.linalg.svd(data_per_class[i], full_matrices=False)
    u1_temp=u1_temp/np.linalg.norm(u1_temp, ord=2, axis=0)
    U_per_class[i] = u1_temp[:, 0:K]
    

Class 0
Class 1
Class 2
Class 3
Class 4
Class 5
Class 6
Class 7
Class 8
Class 9


In [12]:
# clients_idxs = np.arange(10)
# adj_mat = calculating_adjacency(clients_idxs, U_per_class)
# adj_mat = ((180/np.pi)/100)*adj_mat

num = nclass

sim_angle_min = np.zeros([num, num])
sim_angle_tr = np.zeros([num, num])

for i in range(num):
    for j in range(num):
        #kk = selected_clients[i]
        #ll = selected_clients[j]
        F, G = Eq_Basis (U_per_class[i],U_per_class[j])
        #F = copy.deepcopy(U_per_class[i])
        #G = copy.deepcopy(U_per_class[j])
        F_in_G = np.clip(F.T@G, a_min = -1, a_max = +1)

        Angle = np.arccos(np.abs(F_in_G))
        sim_angle_min[i,j] =  (180/np.pi)*np.min(Angle) 
        sim_angle_tr[i,j] =(180/np.pi)*np.trace(Angle)

        #sim_mat[i,j] =  100*np.min(Angle) 


/home/mahdi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in arccos
  


In [13]:
print(sim_angle_min.round(decimals=2))

[[ 0.    6.83  6.55  8.65  9.29 10.48 10.78  7.56  6.66  8.99]
 [ 6.83  0.    7.86  9.14  9.83 11.18  9.64  8.23  6.89  5.73]
 [ 6.55  7.86  0.    4.14  3.69  6.36  5.41  3.66 10.46 11.28]
 [ 8.65  9.14  4.14  0.    3.34  3.79  3.85  4.82 12.44 13.05]
 [ 9.29  9.83  3.69  3.34  0.    4.5   3.38  4.65 13.03 13.64]
 [10.48 11.18  6.36  3.79  4.5   0.    5.64  7.43 13.48 15.29]
 [10.78  9.64  5.41  3.85  3.38  5.64  0.    5.4  13.82 13.33]
 [ 7.56  8.23  3.66  4.82  4.65  7.43  5.4   0.   11.47 10.67]
 [ 6.66  6.89 10.46 12.44 13.03 13.48 13.82 11.47  0.    7.44]
 [ 8.99  5.73 11.28 13.05 13.64 15.29 13.33 10.67  7.44  0.  ]]


In [14]:
print(sim_angle_tr.round(decimals=1))

[[90.  20.5 19.6 25.9 27.9 31.5 32.3 22.7 20.  27. ]
 [20.5 90.  23.6 27.4 29.5 33.5 28.9 24.7 20.7 17.2]
 [19.6 23.6 90.  12.4 11.1 19.1 16.2 11.  31.4 33.8]
 [25.9 27.4 12.4  0.  10.  11.4 11.5 14.5 37.3 39.1]
 [27.9 29.5 11.1 10.   0.  13.5 10.1 14.  39.1 40.9]
 [31.5 33.5 19.1 11.4 13.5  0.  16.9 22.3 40.4 45.9]
 [32.3 28.9 16.2 11.5 10.1 16.9 90.  16.2 41.5 40. ]
 [22.7 24.7 11.  14.5 14.  22.3 16.2 90.  34.4 32. ]
 [20.  20.7 31.4 37.3 39.1 40.4 41.5 34.4  0.  22.3]
 [27.  17.2 33.8 39.1 40.9 45.9 40.  32.  22.3  0. ]]


In [18]:
th = 100
clusters = hierarchical_clustering(copy.deepcopy(sim_angle_min), thresh=th, linkage=args.linkage)
print(clusters)

[[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]


In [41]:
th = 100
flag=True

old_clusters = hierarchical_clustering(copy.deepcopy(sim_angle_min), thresh=th, linkage=args.linkage)

while flag:
    th/=1.1
    new_clusters = hierarchical_clustering(copy.deepcopy(sim_angle_min), thresh=th, linkage=args.linkage)
    print(th)
    print(f'New {new_clusters}')
    print(f'Old {old_clusters}')
    if abs(len(new_clusters)-len(old_clusters)) > 1:
        flag=False
    old_clusters = new_clusters
 
print('EndofWhile')
print(th)
print(f'New {new_clusters}')
print(f'Old {old_clusters}')

90.9090909090909
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
82.64462809917354
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
75.13148009015777
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
68.30134553650706
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
62.0921323059155
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
56.44739300537772
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
51.31581182307065
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
46.650738020973314
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
42.40976183724847
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
38.55432894295315
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
35.04938994813922
New [[0, 8, 1, 9, 2, 7, 3, 4, 6, 5]]
Old [[0, 8, 1, 9, 2, 7, 3,

In [42]:
10*180/(np.pi*100)

5.729577951308232